In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload


import sys, os, inspect
import sys
sys.path.append('/Users/emigardiner/VICO/pjams-ionization/pjams/')

from zeusmp_snapshot_reader import read_zeusmp_snapshot
from zeusmp_snapshot_reader import ScaleFactors
from snapshot import snapshot 
from basic_snapshot import basic_snapshot
import plot
import fluxprof

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm
from matplotlib.colors import TwoSlopeNorm

from PIL import Image, ImageDraw, ImageFont
from IPython.display import Image as DisplayImage

In [ ]:
VICO_loc = '/Users/emigardiner/VICO/pjams-ionization'
MYFONT = ImageFont.truetype('/System/Library/Fonts/Supplemental/Times New Roman.ttf', 65)
SMALLFONT = ImageFont.truetype('/System/Library/Fonts/Supplemental/Times New Roman.ttf', 55)
    

# INPUTS
FREQS = np.array([.01, .05, .1, .5, 1, 5.3, 23, 43, 100, 230 ]) # GHz
FREQS *= 10**9 # Hz

r_kpc = 1
heights_and_scales = np.load(VICO_loc+'/Data/heights_and_scales.npz')
scales = heights_and_scales['scales'] # AU
heights = heights_and_scales['heights'] # AU  

colors = plot.COLORS

# Read in snaps

In [ ]:
nums = np.array([4, 9, 21, 39, 54, 68, 94])
years = np.array(['4,000 yrs', '9,000 yrs', '21,000 yrs', '39,000 yrs', '54,000 yrs', '68,000 yrs', '94,000 yrs'])
masss = np.array([r'1.4 M$_\odot$', r'2 M$_\odot$', r'4 M$_\odot$',
                   r'8 M$_\odot$', r'12 M$_\odot$', r'16 M$_\odot$',
                   r'24 M$_\odot$'])

In [ ]:
snaps = np.empty_like(nums, dtype=snapshot)
for ii, num in enumerate(nums):
    snaps[ii] = snapshot(snap=num, name = ('Snap%03d_n' % num), read_zeusmp = False)
    # shot[ii].load_shock_variables()
    snaps[ii].load_fluxes()

# Load Ionization Fractions

In [ ]:
def load_average_ionfrac_array(Snap, debug=False):
    data_path = (VICO_loc+'/Data/'+Snap.name+'/')
    loaded_average_ionfracs = np.load((data_path+Snap.name+'_average_ionfrac_arrays.npz'))
    Snap.ionfrac_mass = loaded_average_ionfracs['ionfrac_mass']
    Snap.ionfrac_vol =  loaded_average_ionfracs['ionfrac_vol']
    Snap.ionfrac_emis = loaded_average_ionfracs['ionfrac_emis']
    if debug: print(Snap.name + ' ionfrac arrays loaded')
    
def load_average_ionfrac_array_ratio(Snap, const=False, debug=False):
    data_path = (VICO_loc+'/Data/'+Snap.name+'/')
    loaded_average_ionfracs = np.load((data_path+Snap.name+'_average_ionfrac_arrays_const'+str(const)+'.npz'))
    Snap.ionfrac_mass_ratio = loaded_average_ionfracs['ionfrac_mass']
    Snap.ionfrac_vol_ratio =  loaded_average_ionfracs['ionfrac_vol']
    Snap.ionfrac_emis_ratio = loaded_average_ionfracs['ionfrac_emis']
    if debug: print(Snap.name + ' ionfrac_ratio arrays loaded')

In [ ]:
for ss in range(0,len(snaps)):
    load_average_ionfrac_array(snaps[ss])
    load_average_ionfrac_array_ratio(snaps[ss], const=False)
    
# Maximum Velocities for Contours
v_mins = np.array([1,10,100])
def max_velocity_proj_array(Snap, v_min):
    Snap.maxv1 = np.zeros((len(Snap.x1), len(Snap.x2)))
    for ii in range(len(Snap.x1)):
        for jj in range(len(Snap.x2)):
            Snap.maxv1[ii,jj] = np.max(Snap.q['v1'][ii,jj,:])
for ss in range(len(snaps)): 
    for vv in v_mins:
        max_velocity_proj_array(snaps[ss], vv)

# Plot

## new cmap

In [ ]:
import matplotlib.colors as mplcols

def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = mplcols.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

cmap = plt.get_cmap('jet')
new_cmap = truncate_colormap(cmap, 0.1, 1)


## plotting functions

In [ ]:
# Ionization Fraction Methods
import matplotlib.colors as colors
saveloc = '/Users/emigardiner/VICO/pjams-ionization/pillowfiles/ionfrac_fig'

def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

cmap = plt.get_cmap('jet')
new_cmap = truncate_colormap(cmap, 0.1, 1)

mass pcolormesh

In [ ]:
def mass_ionfrac_pcolormesh(
        Snap, year, scale, vmin, vmax, v, cmap=new_cmap, 
        saveloc=saveloc, show_cbar= False, cbar_pad=.2, vertical_cbar=False, 
        show_xlabels=True, show_ylabels=True, show_contours=False, show_legend=False,
        name=None, xx_name=0.02, yy_name=0.22, namecolor='white', namefs=13):
    Snap.X1_2v, Snap.X2_1v = np.meshgrid(Snap.x1, Snap.x2)
    v_min=v_mins[v]
            
    # Make projection log plots for snapshot
    fig = plt.figure(figsize = (4,4))
    fig.tight_layout()
    ax = plt.subplot()
    ax.set_aspect(1)
    ax.set_xlim(-scale/2, scale/2)
    ax.set_ylim(0, scale)
    ax.set_facecolor('black')
    
    cont = ax.pcolormesh(Snap.X2_1v , Snap.X1_2v, np.rot90(np.log10(Snap.ionfrac_mass[v,:,:])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
    
    if(show_xlabels):
        ax.set_xticks([-scale*.25, 0, scale*.25])
        ax.set_xlabel('x (au)', fontsize=28)
        ax.tick_params(axis='x', labelsize=20)
    else:
        ax.tick_params(bottom=False)
        ax.set_xticks([])
    if(show_ylabels):
        ax.set_yticks([scale*.25,scale*.5,scale*.75])
        ax.set_ylabel('z (au)', fontsize=28)
        ax.tick_params(axis='y', labelsize=20)
    else:
        ax.tick_params(left=False)
        ax.set_yticks([])
    if(show_cbar):
        if(vertical_cbar):
            fig.colorbar(cont, label = r'log10($\langle\chi_\mathrm{H+}\rangle$ )', 
                 orientation = 'vertical', fraction=0.038, pad=cbar_pad, extend='min').ax.tick_params(rotation=45)
        else:
            fig.colorbar(cont, label = (r'log10($\langle\chi_\mathrm{H+}\rangle$ )'), 
                     orientation = 'horizontal', fraction=0.038, pad=cbar_pad, extend='min').ax.tick_params(rotation=45)
    
    if(show_contours):
        cntr = ax.contour(Snap.X2_1v , Snap.X1_2v, np.rot90(Snap.maxv1), 
                       levels = [1,10,100], colors = ['darkorange', 'red', 'magenta']) #linewidths = [1, 1.5, 1])
        h,_ = cntr.legend_elements()
        if(show_legend):
            ax.legend([h[0], h[1], h[2]], ['1 km/s', '10km/s', '100km/s'], loc = 'upper center', bbox_to_anchor=(.5, -.65))
    
    if name is not None:
        ax.text(xx_name, yy_name, name, transform=ax.transAxes, fontsize = namefs, color=namecolor, weight='bold')

    if (saveloc is not None): 
        filename = saveloc+'/'+Snap.name+'_ionfrac_mass_v'+str(v_mins[v])+'kms_'+str(scale)+'AU.png'
        fig.savefig(filename, bbox_inches='tight')
        return filename
        

        
def mass_ionfrac_const_pcolormesh(
        Snap, year, const, scale, vmin, vmax, v, cmap=new_cmap, saveloc=saveloc, 
        show_cbar= False, cbar_pad=.2, show_freq=False, vertical_cbar=False, 
        show_xlabels=True, show_ylabels=True, show_contours=False, show_legend=False,
        name=None, xx_name=0.02, yy_name=0.22, namecolor='white', namefs=13):
    Snap.X1_2v, Snap.X2_1v = np.meshgrid(Snap.x1, Snap.x2)
    v_min=v_mins[v]
            
    # Make projection log plots for snapshot
    fig = plt.figure(figsize = (4,4))
    fig.tight_layout()
    ax = plt.subplot()
    ax.set_aspect(1)
    ax.set_xlim(-scale/2, scale/2)
    ax.set_ylim(0, scale)
    ax.set_facecolor('black')
    
    if(const):
        cont = ax.pcolormesh(Snap.X2_1v , Snap.X1_2v, np.rot90(np.log10(Snap.ionfrac_mass_const[v,:,:])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
    else: 
        cont = ax.pcolormesh(Snap.X2_1v , Snap.X1_2v, np.rot90(np.log10(Snap.ionfrac_mass_ratio[v,:,:])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
    if(show_xlabels):
        ax.set_xticks([-scale*.25, 0, scale*.25])
        ax.set_xlabel('x (au)', fontsize=28)
        ax.tick_params(axis='x', labelsize=20)
    else:
        ax.tick_params(bottom=False)
        ax.set_xticks([])
    if(show_ylabels):
        ax.set_yticks([scale*.25,scale*.5,scale*.75])
        ax.set_ylabel('z (au)', fontsize=28)
        ax.tick_params(axis='y', labelsize=20)
    else:
        ax.tick_params(left=False)
        ax.set_yticks([])
    if(show_cbar):
        if(vertical_cbar):
            fig.colorbar(cont, label = r'log10($\langle\chi_\mathrm{H+}\rangle$ )', 
                 orientation = 'vertical', fraction=0.038, pad=cbar_pad, extend='min').ax.tick_params(rotation=45)
        else:
            fig.colorbar(cont, label = (r'log10($\langle\chi_\mathrm{H+}\rangle$ )'), 
                     orientation = 'horizontal', fraction=0.038, pad=cbar_pad, extend='min').ax.tick_params(rotation=45)
    
    if(show_contours):
        cntr = ax.contour(Snap.X2_1v , Snap.X1_2v, np.rot90(Snap.maxv1), 
                       levels = [1,10,100], colors = ['darkorange', 'red', 'magenta']) #linewidths = [1, 1.5, 1])
        h,_ = cntr.legend_elements()
        if(show_legend):
            ax.legend([h[0], h[1], h[2]], ['1 km/s', '10km/s', '100km/s'], loc = 'upper center', bbox_to_anchor=(.5, -.65))
    
    if name is not None:
        ax.text(xx_name, yy_name, name, transform=ax.transAxes, fontsize = namefs, color=namecolor, weight='bold')

    if ((saveloc) and (const)): 
        filename = saveloc+'/'+Snap.name+'_ionfrac_mass_const_v'+str(v_mins[v])+'kms_'+str(scale)+'AU.png'
        fig.savefig(filename, bbox_inches='tight')
        return filename
    elif ((saveloc)):  #  and (!const)
        filename = saveloc+'/'+Snap.name+'_ionfrac_mass_ratio_v'+str(v_mins[v])+'kms_'+str(scale)+'AU.png'
        fig.savefig(filename, bbox_inches='tight')
        return filename
   

vol pcolormesh

In [ ]:
     
        
def vol_ionfrac_pcolormesh(Snap, year, scale, vmin, vmax, v, cmap=new_cmap,
                            saveloc=saveloc, show=False, show_cbar= False, 
                            show_xlabels=True, show_ylabels=True, 
                            cbar_pad=.2, show_freq=False, vertical_cbar=False, 
                            show_contours=False, show_legend=False):
    Snap.X1_2v, Snap.X2_1v = np.meshgrid(Snap.x1, Snap.x2)
#     print(v)
    v_min=v_mins[v]

    # Make projection log plots for snapshot
    fig = plt.figure(figsize = (4,4))
    fig.tight_layout()
    ax = plt.subplot()
    ax.set_aspect(1)
    ax.set_xlim(-scale/2, scale/2)
    ax.set_ylim(0, scale)
    ax.set_facecolor('black')
    
    cont = ax.pcolormesh(Snap.X2_1v , Snap.X1_2v, np.rot90(np.log10(Snap.ionfrac_vol[v,:,:])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
    if(show_xlabels):
        ax.set_xticks([-scale*.25, 0, scale*.25])
        ax.set_xlabel('x (au)', fontsize=28)
        ax.tick_params(axis='x', labelsize=20)
    else:
        ax.tick_params(bottom=False)
        ax.set_xticks([])
    if(show_ylabels):
        ax.set_yticks([scale*.25,scale*.5,scale*.75])
        ax.set_ylabel('z (au)', fontsize=28)
        ax.tick_params(axis='y', labelsize=20)
    else:
        ax.tick_params(left=False)
        ax.set_yticks([])
    if(show_cbar):
        if(vertical_cbar):
            fig.colorbar(cont, label = r'log10($\langle\chi_\mathrm{H+}\rangle$ )', 
                 orientation = 'vertical', fraction=0.038, pad=cbar_pad, extend='min').ax.tick_params(rotation=45)
        else:
            fig.colorbar(cont, label = (r'log10($\langle\chi_\mathrm{H+}\rangle$ )'), 
                     orientation = 'horizontal', fraction=0.038, pad=cbar_pad, extend='min').ax.tick_params(rotation=45)
      
    if(show_contours):
        cntr = ax.contour(Snap.X2_1v , Snap.X1_2v, np.rot90(Snap.maxv1), 
                       levels = [1,10,100], colors = ['darkorange', 'red', 'magenta']) #linewidths = [1, 1.5, 1])
        h,_ = cntr.legend_elements()
        if(show_legend):
            ax.legend([h[0], h[1], h[2]], ['1 km/s', '10km/s', '100km/s'], loc = 'upper center', bbox_to_anchor=(.5, -.65))
    
#     ax.text(.02, .22, (r'$\mathbf{\langle\chi_{H+}\rangle, vol}$'+'\n'+year+'\n'+'$\mathbf{v_{min}}$='+str(v_min)+' km/s'), ha='left', va='top', 
#             transform=ax.transAxes, fontsize = 13, color='white', weight='bold')
    if (saveloc != False): 
        filename = saveloc+'/'+Snap.name+'_ionfrac_vol_v'+str(v_mins[v])+'kms_'+str(scale)+'AU.png'
        fig.savefig(filename, bbox_inches='tight')
        return filename

def vol_ionfrac_const_pcolormesh(Snap, year, const, scale, vmin, vmax, v, cmap=new_cmap,
                            saveloc=saveloc, show=False, show_cbar= False, 
                            show_xlabels=True, show_ylabels=True, 
                            cbar_pad=.2, show_freq=False, vertical_cbar=False, 
                            show_contours=False, show_legend=False):
    Snap.X1_2v, Snap.X2_1v = np.meshgrid(Snap.x1, Snap.x2)
#     print(v)
    v_min=v_mins[v]

    # Make projection log plots for snapshot
    fig = plt.figure(figsize = (4,4))
    fig.tight_layout()
    ax = plt.subplot()
    ax.set_aspect(1)
    ax.set_xlim(-scale/2, scale/2)
    ax.set_ylim(0, scale)
    ax.set_facecolor('black')
    if(const):
        cont = ax.pcolormesh(Snap.X2_1v , Snap.X1_2v, np.rot90(np.log10(Snap.ionfrac_vol_const[v,:,:])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
    else:
        cont = ax.pcolormesh(Snap.X2_1v , Snap.X1_2v, np.rot90(np.log10(Snap.ionfrac_vol_ratio[v,:,:])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
    if(show_xlabels):
        ax.set_xticks([-scale*.25, 0, scale*.25])
        ax.set_xlabel('x (au)', fontsize=28)
        ax.tick_params(axis='x', labelsize=20)
    else:
        ax.tick_params(bottom=False)
        ax.set_xticks([])
    if(show_ylabels):
        ax.set_yticks([scale*.25,scale*.5,scale*.75])
        ax.set_ylabel('z (au)', fontsize=28)
        ax.tick_params(axis='y', labelsize=20)
    else:
        ax.tick_params(left=False)
        ax.set_yticks([])
    if(show_cbar):
        if(vertical_cbar):
            fig.colorbar(cont, label = r'log10($\langle\chi_\mathrm{H+}\rangle$ )', 
                 orientation = 'vertical', fraction=0.038, pad=cbar_pad, extend='min').ax.tick_params(rotation=45)
        else:
            fig.colorbar(cont, label = (r'log10($\langle\chi_\mathrm{H+}\rangle$ )'), 
                     orientation = 'horizontal', fraction=0.038, pad=cbar_pad, extend='min').ax.tick_params(rotation=45)
      
    if(show_contours):
        cntr = ax.contour(Snap.X2_1v , Snap.X1_2v, np.rot90(Snap.maxv1), 
                       levels = [1,10,100], colors = ['darkorange', 'red', 'magenta']) #linewidths = [1, 1.5, 1])
        h,_ = cntr.legend_elements()
        if(show_legend):
            ax.legend([h[0], h[1], h[2]], ['1 km/s', '10km/s', '100km/s'], loc = 'upper center', bbox_to_anchor=(.5, -.65))
    
#     ax.text(.02, .22, (r'$\mathbf{\langle\chi_{H+}\rangle, vol}$'+'\n'+year+'\n'+'$\mathbf{v_{min}}$='+str(v_min)+' km/s'), ha='left', va='top', 
#             transform=ax.transAxes, fontsize = 13, color='white', weight='bold')
    if ((saveloc) and (const)): 
        filename = saveloc+'/'+Snap.name+'_ionfrac_vol_const_v'+str(v_mins[v])+'kms_'+str(scale)+'AU.png'
        fig.savefig(filename, bbox_inches='tight')
        return filename
    elif ((saveloc)):  #  and (!const)
        filename = saveloc+'/'+Snap.name+'_ionfrac_vol_ratio_v'+str(v_mins[v])+'kms_'+str(scale)+'AU.png'
        fig.savefig(filename, bbox_inches='tight')
        return filename
 

emis pcolormesh

In [ ]:
   
        
def emis_ionfrac_pcolormesh(Snap, year, scale, f, vmin, vmax, v, cmap=new_cmap,
                            saveloc=saveloc, show=False, show_cbar= False, 
                            show_xlabels=True, show_ylabels=True, 
                            cbar_pad=.2, show_freq=False, vertical_cbar=False, 
                            show_contours=False, show_legend=False, 
                           with_delta = True):
    Snap.X1_2v, Snap.X2_1v = np.meshgrid(Snap.x1, Snap.x2)
    v_min=v_mins[v]
    nu=FREQS[f]
           
    # Make projection log plots for snapshot
    fig = plt.figure(figsize = (4,4))
    fig.tight_layout()
    ax = plt.subplot()
    ax.set_aspect(1)
    ax.set_xlim(-scale/2, scale/2)
    ax.set_ylim(0, scale)
    ax.set_facecolor('black')
    
    cont = ax.pcolormesh(Snap.X2_1v , Snap.X1_2v, np.rot90(np.log10(Snap.ionfrac_emis[f,v,:,:])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
#     ax.plot(Snap.x2[9], Snap.x1[0], 
#             color='cyan', markersize=50, marker='+')
#     ax.plot(Snap.x2[9], Snap.x1[165], 
#             color='cyan', markersize=50, marker='+')
#     ax.plot(Snap.x2[270], Snap.x1[0], 
#             color='cyan', markersize=50, marker='+')
#     ax.plot(Snap.x2[270], Snap.x1[165], 
#             color='cyan', markersize=50, marker='+')

    if(show_xlabels):
        ax.set_xticks([-scale*.25, 0, scale*.25])
        ax.set_xlabel('x (au)', fontsize=28)
        ax.tick_params(axis='x', labelsize=20)
    else:
        ax.tick_params(bottom=False)
        ax.set_xticks([])
    if(show_ylabels):
        ax.set_yticks([scale*.25,scale*.5,scale*.75])
        ax.set_ylabel('z (au)', fontsize=28)
        ax.tick_params(axis='y', labelsize=20)
    else:
        ax.tick_params(left=False)
        ax.set_yticks([])
    if(show_cbar):
        if(vertical_cbar):
            fig.colorbar(cont, label = r'log10($\langle\chi_\mathrm{H+}\rangle$ )', 
                 orientation = 'vertical', fraction=0.038, pad=cbar_pad, extend='min').ax.tick_params(rotation=45)
        else:
            fig.colorbar(cont, label = (r'log10($\langle\chi_\mathrm{H+}\rangle$ )'), 
                     orientation = 'horizontal', fraction=0.038, pad=cbar_pad, extend='min').ax.tick_params(rotation=45)   
    
    if(show_contours):
        cntr = ax.contour(Snap.X2_1v , Snap.X1_2v, np.rot90(Snap.maxv1), 
                       levels = [1,10,100], colors = ['darkorange', 'red', 'magenta']) #linewidths = [1, 1.5, 1])
        h,_ = cntr.legend_elements()
        if(show_legend):
            ax.legend([h[0], h[1], h[2]], ['1 km/s', '10km/s', '100km/s'], loc = 'upper center', bbox_to_anchor=(.5, -.65))
    
#     ax.text(.02, .22, (r'$\mathbf{\langle\chi_{H+}\rangle, emis}$'+'\n'+year+'\n'+'$\mathbf{v_{min}}$='+str(v_min)+' km/s'), ha='left', va='top', 
#             transform=ax.transAxes, fontsize = 13, color='white', weight='bold')
    if (saveloc != False): 
        filename = saveloc+'/'+Snap.name+'_ionfrac_emis_v'+str(v_mins[v])+'kms_'+str(scale)+'AU.png'
        fig.savefig(filename, bbox_inches='tight')
        return filename
    
        
def emis_ionfrac_const_pcolormesh(Snap, year, const, scale, f, vmin, vmax, v, cmap=new_cmap,
                            saveloc=saveloc, show=False, show_cbar= False, 
                            show_xlabels=True, show_ylabels=True, 
                            cbar_pad=.2, show_freq=False, vertical_cbar=False, 
                            show_contours=False, show_legend=False, 
                           with_delta = True):
    Snap.X1_2v, Snap.X2_1v = np.meshgrid(Snap.x1, Snap.x2)
    v_min=v_mins[v]
    nu=FREQS[f]
           
    # Make projection log plots for snapshot
    fig = plt.figure(figsize = (4,4))
    fig.tight_layout()
    ax = plt.subplot()
    ax.set_aspect(1)
    ax.set_xlim(-scale/2, scale/2)
    ax.set_ylim(0, scale)
    ax.set_facecolor('black')
    
    if(const):
        cont = ax.pcolormesh(Snap.X2_1v , Snap.X1_2v, np.rot90(np.log10(Snap.ionfrac_emis_const[f,v,:,:])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
    else:
        cont = ax.pcolormesh(Snap.X2_1v , Snap.X1_2v, np.rot90(np.log10(Snap.ionfrac_emis_ratio[f,v,:,:])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
    if(show_xlabels):
        ax.set_xticks([-scale*.25, 0, scale*.25])
        ax.set_xlabel('x (au)', fontsize=28)
        ax.tick_params(axis='x', labelsize=20)
    else:
        ax.tick_params(bottom=False)
        ax.set_xticks([])
    if(show_ylabels):
        ax.set_yticks([scale*.25,scale*.5,scale*.75])
        ax.set_ylabel('z (au)', fontsize=28)
        ax.tick_params(axis='y', labelsize=20)
    else:
        ax.tick_params(left=False)
        ax.set_yticks([])
    if(show_cbar):
        if(vertical_cbar):
            fig.colorbar(cont, label = r'log10($\langle\chi_\mathrm{H+}\rangle$ )', 
                 orientation = 'vertical', fraction=0.038, pad=cbar_pad, extend='min').ax.tick_params(rotation=45)
        else:
            fig.colorbar(cont, label = (r'log10($\langle\chi_\mathrm{H+}\rangle$ )'), 
                     orientation = 'horizontal', fraction=0.038, pad=cbar_pad, extend='min').ax.tick_params(rotation=45)   
    
    if(show_contours):
        cntr = ax.contour(Snap.X2_1v , Snap.X1_2v, np.rot90(Snap.maxv1), 
                       levels = [1,10,100], colors = ['darkorange', 'red', 'magenta']) #linewidths = [1, 1.5, 1])
        h,_ = cntr.legend_elements()
        if(show_legend):
            ax.legend([h[0], h[1], h[2]], ['1 km/s', '10km/s', '100km/s'], loc = 'upper center', bbox_to_anchor=(.5, -.65))
    
#     ax.text(.02, .22, (r'$\mathbf{\langle\chi_{H+}\rangle, emis}$'+'\n'+year+'\n'+'$\mathbf{v_{min}}$='+str(v_min)+' km/s'), ha='left', va='top', 
#             transform=ax.transAxes, fontsize = 15, color='white', weight='bold')
    if ((saveloc) and (const)): 
        filename = saveloc+'/'+Snap.name+'_ionfrac_emis_const_v'+str(v_mins[v])+'kms_'+str(scale)+'AU.png'
        fig.savefig(filename, bbox_inches='tight')
        return filename
    elif ((saveloc)):  #  and (!const)
        filename = saveloc+'/'+Snap.name+'_ionfrac_emis_ratio_v'+str(v_mins[v])+'kms_'+str(scale)+'AU.png'
        fig.savefig(filename, bbox_inches='tight')
        return filename
    


# Make Small Plot Arrays

In [ ]:
nrows = len(snaps) - 1
ncols = 9
scale = 25000
imgfiles_arr_25000au_noratio = np.empty((nrows, ncols), dtype=object)
r=0
for s in range(1,len(snaps)):
    c=0
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        if(v==0): 
            show_ylabels = True
            name = masss[s]
        else: 
            show_ylabels = False
            name = None
        filename = mass_ionfrac_pcolormesh(snaps[s], years[s], scale, vmin=-3, vmax=0, v=v,
                               show_xlabels=show_xlabels, show_ylabels=show_ylabels,
                               name=name, xx_name=0.02, yy_name=0.05, namefs=30)
        imgfiles_arr_25000au_noratio[r,c] = filename
        c+=1
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        show_ylabels = False
        filename = vol_ionfrac_pcolormesh(snaps[s], years[s], scale, vmin=-3, vmax=0, v=v,
                              show_xlabels=show_xlabels, show_ylabels=show_ylabels)
        imgfiles_arr_25000au_noratio[r,c] = filename
        c+=1
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        show_ylabels = False
        filename = emis_ionfrac_pcolormesh(snaps[s], years[s], scale, f=5, vmin=-3, vmax=0, v=v,
                               show_xlabels=show_xlabels, show_ylabels=show_ylabels)
        imgfiles_arr_25000au_noratio[r,c] = filename
        c+=1
    r+=1

## PIL Image Methods

In [ ]:
def ionfrac_header(allimage, side, leftax, header): 
   # add header
    draw = ImageDraw.Draw(allimage)
    myFont = MYFONT 
    smallFont = SMALLFONT

    mname = "Mass-Weighted"
    # mw, mh = draw.textsize(mname)
    vname = "Volume-Weighted"
    # vw, vh = draw.textsize(mname)
    ename = "Emission-Weighted"
    # ew, eh = draw.textsize(mname)

    t1 = leftax+(1.5*side)
    t2 = t1+3*side
    t3 = t2+3*side

    draw.text((t1,0), mname, font=myFont, fill=(0,0,0), anchor='mt')
    draw.text((t2,0), vname, font=myFont, fill=(0,0,0), anchor='mt')
    draw.text((t3,0), ename, font=myFont, fill=(0,0,0), anchor='mt')

    draw.text((leftax+60,header-60), "1 km/s", font=smallFont, fill=(26,38,134))
    draw.text((leftax+40+side,header-60), "10 km/s", font=smallFont, fill=(26,38,134))
    draw.text((leftax+30+2*side,header-60), "100 km/s", font=smallFont, fill=(26,38,134))
    draw.text((leftax+60+3*side,header-60), "1 km/s", font=smallFont, fill=(26,38,134))
    draw.text((leftax+40+4*side,header-60), "10 km/s", font=smallFont, fill=(26,38,134))
    draw.text((leftax+30+5*side,header-60), "100 km/s", font=smallFont, fill=(26,38,134))
    draw.text((leftax+60+6*side,header-60), "1 km/s", font=smallFont, fill=(26,38,134))
    draw.text((leftax+40+7*side,header-60), "10 km/s", font=smallFont, fill=(26,38,134))
    draw.text((leftax+30+8*side,header-60), "100 km/s", font=smallFont, fill=(26,38,134))
    return allimage


def ionfrac_section_breaks(allimage, side, leftax, header):
    # add section breaks
    line1 = [(leftax+3*side,0), (leftax+3*side, header)]
    line2 = [(leftax+6*side,0), (leftax+6*side, header)]
    draw = ImageDraw.Draw(allimage)
    draw.line(line1,fill=(0,0,0),width=5)
    draw.line(line2,fill=(0,0,0),width=5)
    return allimage


def pil_image(imgfiles, debug=True, head=0):
    nrows = len(imgfiles)
    ncols = len(imgfiles[0])
    if debug: print(f"{nrows=}, {ncols=}")

    # get side lengths
    corner_img = Image.open(imgfiles[nrows-1, 0])
    middle_img = Image.open(imgfiles[0,1])
    side = middle_img.width
    left = corner_img.width - middle_img.width
    bott = corner_img.height - middle_img.height
    if debug: print(f"{corner_img.size=}, {middle_img.size=}")
    if debug: print(f"{side=}, {left=}, {bott=}, {head=}")

    # make allimage
    allimage=Image.new("RGBA", (ncols*side+left, nrows*side+bott+head))

    # paste mini images
    yy = head
    for rr in range(nrows):
        xx = 0 
        for cc in range(ncols):
            if debug: print(f"{rr=}, {cc=}, {xx=}, {yy=}")
            img = Image.open(imgfiles[rr,cc])
            allimage.paste(img, ((xx, yy)))
            if cc==0: xx += left+side
            else: xx += side
        yy += side
    
    details = {
        'nrows':nrows, 'ncols':ncols, 'side':side, 'left':left, 'bott':bott, 'head':head
    }
    return allimage, details

def add_mass_labels(allimage, side, left, head, labels, font=MYFONT, color=(255, 255, 255), 
                    debug=False):
    draw = ImageDraw.Draw(allimage)
    
    xx = left+5
    yy = head+side-5
    for label in labels:
        if debug: print('adding %s label' % label)
        draw.text((xx,yy), label, font=font, fill=color)
        yy += side
    return allimage

def ionfrac_pil_image(imgfiles, debug=False):
    allimage, details = pil_image(imgfiles, head=150, debug=debug)
    side = details['side']
    left = details['left']
    head = details['head']
    allimage = ionfrac_header(allimage, side, left, head)
    allimage = ionfrac_section_breaks(allimage, side, left, head)
    # allimage = add_mass_labels(allimage, side, left, head, labels, debug=debug)
    return allimage

#### 2.2.1 PIL Figure: 25000au no ratio

In [ ]:
allimage = ionfrac_pil_image(imgfiles_arr_25000au_noratio, debug=False)
display(allimage)
allimage.save(VICO_loc+'/figures/ionfrac_flooded_25000au.png')

#### 2.2.2 PIL Figure: 4000au no ratio

In [ ]:
nrows = len(snaps) - 1
ncols = 9
scale = 4000
imgfiles_arr_4000au_noratio = np.empty((nrows, ncols), dtype=object)
r=0
for s in range(1,len(snaps)):
    c=0
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        if(v==0): 
            show_ylabels = True
            name = masss[s]
        else: 
            show_ylabels = False
            name = None
        filename = mass_ionfrac_pcolormesh(snaps[s], years[s], scale, vmin=-3, vmax=0, v=v,
                               show_xlabels=show_xlabels, show_ylabels=show_ylabels,
                               name=name, xx_name=0.02, yy_name=0.05, namefs=30)
        imgfiles_arr_25000au_noratio[r,c] = filename
        c+=1
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        show_ylabels = False
        filename = vol_ionfrac_pcolormesh(snaps[s], years[s], scale, vmin=-3, vmax=0, v=v,
                              show_xlabels=show_xlabels, show_ylabels=show_ylabels)
        imgfiles_arr_25000au_noratio[r,c] = filename
        c+=1
    for v in range(len(v_mins)):
        if(s==len(snaps)-1): show_xlabels = True
        else: show_xlabels = False
        show_ylabels = False
        filename = emis_ionfrac_pcolormesh(snaps[s], years[s], scale, f=5, vmin=-3, vmax=0, v=v,
                               show_xlabels=show_xlabels, show_ylabels=show_ylabels)
        imgfiles_arr_25000au_noratio[r,c] = filename
        c+=1
    r+=1

In [ ]:
side = 328
left = 122
bott = 79
head = 150

t1 = side/2+left
t2 = t1+side
t3 = t2+side
print(f"{t1=}, {t2=}, {t3=}")

In [ ]:
allimage = ionfrac_pil_image(imgfiles_arr_4000au_noratio, debug=False)
display(allimage)
allimage.save(VICO_loc+'/figures/ionfrac_flooded_4000au.png')

In [ ]:
print(allimage.size)

In [ ]:
print(3074/2)